In [1]:
%load_ext autoreload
%autoreload 2
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.molSize = (800, 800)
import numpy as np
import pandas as pd
from toolsets.search import string_search, quick_search_values, quick_search_sorted
from tqdm import tqdm
import os
from toolsets.file_io import get_file_list
from rdkit import Chem
from toolsets.ff_droup import feature_finding, auto_EIC, EIC, get_feature
from toolsets.raw_data_scaffold import read_mzml
from toolsets.std_list_prep import neutrilize_salt, neutrilize_salt_df, std_list_cleanup
from toolsets.std_list_prep import complete_adducts
import toolsets.spectra_operations as so
from toolsets.spectra_plotter import head_to_tail_plot, ms2_plot

import time

In [1]:
# master_dir ='/Volumes/scratch/Fanzhou/polyphenol_lib'
# mzml_dir = '/Volumes/scratch/Fanzhou/polyphenol_lib/mzml/pos'
master_dir ='/Volumes/Samsung_T5/polyphenol_lib/'
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos_high_ce/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos_high_ce/pl'
pos_adducts = ['[M]+','[M+H]+', '[M+Na]+', '[M+NH4]+']
neg_adducts = ['[M]-','[M-H]-','[M+Cl]-', '[M-H2O-H]-', ]

In [8]:
features = pd.read_csv(os.path.join('/Users/fanzhoukong/Downloads/polyphenol_share/pos_high_ce/pl/Polyphenols-pos-mix1_features.csv'))

In [7]:
quick_search_values(features, 'precursor_mz', 183.0651848-0.005*2, 183.0651848-0.005*2)

,precursor_mz,rt_apex,ms1_intensity,rt_start,rt_end,iso_state,is_dut,is_halo,rt_offset,mz_offset,reci_snr,peak_purity,msms,ms1_scan_range,ms2_scan_idx_range_list,ms2_scan_idx,mix


In [10]:
features.sort_values(by = 'precursor_mz')

,precursor_mz,rt_apex,ms1_intensity,rt_start,rt_end,iso_state,is_dut,is_halo,rt_offset,mz_offset,reci_snr,peak_purity,msms,ms1_scan_range,ms2_scan_idx_range_list,ms2_scan_idx,mix
43,80.022287,0.309658,4.395213e+06,0.282792,0.539282,0.0,False,False,0.000819,0.000009,0.021139,0.710530,59.895450592041016\t2524.124267578125\n59.9650...,"[61, 67, 118]","[200, 215]",200,Polyphenols-pos-mix1
12,81.014723,0.309610,1.353449e+07,0.255774,0.530151,0.0,True,False,0.002401,0.000006,0.023171,0.921160,51.42530822753906\t7881.869140625\n54.79056930...,"[55, 67, 116]","[244, 199, 260, 214, 229]",199,Polyphenols-pos-mix1
100,84.957298,0.198811,1.277683e+06,0.173970,0.340846,NaN,False,False,0.005391,0.000031,0.096806,0.970966,56.963409423828125\t29319.28515625\n59.3029785...,"[37, 42, 74]",[130],130,Polyphenols-pos-mix1
48,87.052826,9.503187,2.974140e+06,9.337323,9.800334,0.0,False,False,0.002177,0.000008,0.017218,0.871469,59.058937072753906\t167096.359375\n59.06238174...,"[1974, 2009, 2074]",[6026],6026,Polyphenols-pos-mix1
39,88.109554,9.411881,3.249464e+06,9.192160,9.707337,0.0,False,False,0.002903,0.000033,0.120525,0.915077,53.037353515625\t2550.78759765625\n58.06375885...,"[1945, 1989, 2054]","[5909, 5864, 6011, 5965, 5924, 5879, 5980, 593...",5965,Polyphenols-pos-mix1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,611.293501,2.310534,2.572035e+07,2.281176,2.571276,0.0,False,False,0.008922,0.000031,0.001620,0.992035,53.03732681274414\t8832.166015625\n55.01661300...,"[497, 503, 559]","[1504, 1519, 1534]",1504,Polyphenols-pos-mix1
4,625.194485,2.923807,1.986153e+07,2.890394,3.240985,0.0,False,False,0.003124,0.000066,0.000387,0.970552,53.037227630615234\t66027.5859375\n55.01644134...,"[627, 634, 702]","[1901, 1915]",1901,Polyphenols-pos-mix1
19,642.220520,2.923363,6.765424e+06,2.890394,3.189125,0.0,False,False,0.002119,0.000182,0.000179,0.988302,57.03219223022461\t35605.875\n63.6712760925293...,"[627, 634, 691]",[1903],1903,Polyphenols-pos-mix1
55,647.175720,2.923305,2.649250e+06,2.894496,3.051255,0.0,False,False,0.008454,0.000027,0.000046,0.984397,58.80495834350586\t2978.610107421875\n60.44860...,"[628, 634, 662]",[1907],1907,Polyphenols-pos-mix1


In [12]:
183.0730-183.0651848

0.007815200000010236

In [56]:
# std_list_pos = pd.read_csv(os.path.join(master_dir, 'std_list_pos.csv'))
# std_list_neg = pd.read_csv(os.path.join(master_dir, 'std_list_neg.csv'))
# std_list_nt = neutrilize_salt_df(std_list_pos, smile_column='smiles')
# std_list_cleaned =std_list_cleanup(std_list_nt)
# std_list_pos_adduct = complete_adducts(std_list_cleaned, adducts=pos_adducts)
# std_list_nt = neutrilize_salt_df(std_list_neg, smile_column='smiles')
# std_list_cleaned =std_list_cleanup(std_list_nt)
# std_list_neg_adduct = complete_adducts(std_list_cleaned, adducts=neg_adducts)
# std_list_pos_adduct.to_csv(os.path.join(master_dir, 'std_list_pos_adduct.csv'), index = False)
# std_list_neg_adduct.to_csv(os.path.join(master_dir, 'std_list_neg_adduct.csv'), index = False)

# above is still preparing std list

# Below is for neg

In [148]:
std_list_pos_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_pos_adduct.csv'))
std_list_neg_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_neg_adduct.csv'))

In [158]:
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg/pl'
library_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg/curated_library'
for p in [ library_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [150]:
matched_all = pd.DataFrame()
for mix in std_list_neg_adduct['mix'].unique():
    try:
        matched_temp = pd.read_csv(os.path.join(pl_dir, mix+'_matched.csv'))
        matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    except:
        pass

In [151]:
matched_all['rt_offset']=[abs(x-y) for x, y in zip(matched_all['reference_rt'], matched_all['rt_apex'])]

In [152]:
_tocheck = matched_all.query('rt_offset>0.1')

In [31]:
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg/EICs'

In [36]:
for index, row in tqdm(_tocheck.iterrows(), total = len(_tocheck)):
    if row['comment']=='Major':
        ms1, ms2 = read_mzml(row['reference_mix'], mzml_dir)
        rt_list, intensity_list = auto_EIC(row['reference_mz'],ms1)
        EIC(rt_list, intensity_list, vlines_location_1=[row['reference_rt']], vlines_location_2=[row['rt_apex']],
            savepath=os.path.join(eic_dir, row['reference_name']+'_'+row['reference_adduct']+'.png'), show=False)

100%|██████████| 27/27 [00:53<00:00,  1.98s/it]


In [153]:
from toolsets.denoising_related_functions import denoise_h

In [39]:
reference_db_sorted = pd.read_csv('db/formulaDB_sorted.csv')

In [154]:
msms_denoised = []
ei = []
for index, row in tqdm(matched_all.iterrows(), total = len(matched_all)):
    result_temp = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], reference_db_sorted, mass_error=0.02)
    msms_denoised.append(result_temp[0])
    ei.append(result_temp[1])

100%|██████████| 602/602 [01:58<00:00,  5.07it/s]


In [159]:
matched_all['ei']=ei
matched_all['msms_denoised']=msms_denoised

In [160]:
matched_good_quality = matched_all.query('ei>90 and rt_offset <=0.1')

In [161]:
matched_all.to_csv(os.path.join(library_dir, 'neg_matched_all.csv'), index = False)
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_matched_good_quality.csv'), index = False)
export_library_msp(matched_good_quality, 'neg_matched_good_quality.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)
export_library_msp(matched_all, 'neg_matched_all.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)

In [273]:
matched_all=pd.read_csv(os.path.join(library_dir, 'neg_matched_all.csv'))
matched_good_quality = matched_all.query('ei>90')
rt_matched = []
for index, row in matched_good_quality.iterrows():
    if row['comment']=='Major' and row['rt_offset']>=0.1:
        rt_matched.append(False)
    else:
        rt_matched.append(True)
matched_good_quality['rt_matched']=rt_matched
matched_good_quality=matched_good_quality[matched_good_quality['rt_matched']==True]

In [276]:
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_matched_good_quality.csv'), index = False)

In [ ]:
matched_all.to_csv(os.path.join(library_dir, 'neg_matched_all.csv'), index = False)
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_matched_good_quality.csv'), index = False)
export_library_msp(matched_good_quality, 'neg_matched_good_quality.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)
export_library_msp(matched_all, 'neg_matched_all.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)

In [73]:
from toolsets.file_io import export_library_msp

In [79]:
export_library_msp(matched_good_quality, os.path.join(master_dir, 'neg_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 524/524 [00:00<00:00, 1690.91it/s]


In [80]:
export_library_msp(matched_all, os.path.join(master_dir, 'neg_matched_all.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 602/602 [00:00<00:00, 1698.21it/s]


# pos high ce

In [2]:
611.29-611.3115631

-0.021563099999980295

# neg high ce


In [278]:
std_list_pos_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_pos_adduct.csv'))
std_list_neg_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_neg_adduct.csv'))

In [279]:
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/pl'
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/EICs'
library_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/curated_library'
for p in [eic_dir, library_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [142]:
matched_all = pd.DataFrame()
for mix in std_list_neg_adduct['mix'].unique():
    try:
        matched_temp = pd.read_csv(os.path.join(pl_dir, mix+'_matched.csv'))
        matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    except:
        pass

In [143]:
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/EICs'
library_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/curated_library'
for p in [eic_dir, library_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [144]:
matched_all['rt_offset']=[abs(x-y) for x, y in zip(matched_all['reference_rt'], matched_all['rt_apex'])]
_tocheck = matched_all.query('rt_offset>0.1')
for index, row in tqdm(_tocheck.iterrows(), total = len(_tocheck)):
    if row['comment']=='Major':
        ms1, ms2 = read_mzml(row['reference_mix'], mzml_dir)
        rt_list, intensity_list = auto_EIC(row['reference_mz'],ms1)
        EIC(rt_list, intensity_list, vlines_location_1=[row['reference_rt']], vlines_location_2=[row['rt_apex']],
            savepath=os.path.join(eic_dir, row['reference_name']+'_'+row['reference_adduct']+'.png'), show=False)

100%|██████████| 41/41 [00:58<00:00,  1.42s/it]


In [145]:
msms_denoised = []
ei = []
for index, row in tqdm(matched_all.iterrows(), total = len(matched_all)):
    result_temp = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], reference_db_sorted, mass_error=0.02)
    msms_denoised.append(result_temp[0])
    ei.append(result_temp[1])
matched_all['ei']=ei
matched_all['msms_denoised']=msms_denoised

100%|██████████| 599/599 [02:15<00:00,  4.43it/s]


In [146]:
# matched_good_quality = matched_all.query('ei>90 and rt_offset <=0.1')
# matched_all.to_csv(os.path.join(library_dir, 'neg_high_ce_matched_all.csv'), index = False)
# matched_good_quality.to_csv(os.path.join(library_dir, 'neg_high_ce_matched_good_quality.csv'), index = False)

In [280]:
matched_all=pd.read_csv(os.path.join(library_dir, 'neg_high_ce_matched_all.csv'))
matched_good_quality = matched_all.query('ei>90')
rt_matched = []
for index, row in matched_good_quality.iterrows():
    if row['comment']=='Major' and row['rt_offset']>=0.1:
        rt_matched.append(False)
    else:
        rt_matched.append(True)
matched_good_quality['rt_matched']=rt_matched
matched_good_quality=matched_good_quality[matched_good_quality['rt_matched']==True]

In [281]:
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_high_ce_matched_good_quality.csv'), index = False)

In [284]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'neg_high_ce_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 548/548 [00:00<00:00, 1252.46it/s]


In [147]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'neg_high_ce_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)
export_library_msp(matched_all, os.path.join(library_dir, 'neg_high_ce_matched_all.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 599/599 [00:00<00:00, 1201.53it/s]


# pos

In [287]:
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos/pl'
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/pos/EICs'
library_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos/curated_library'
for p in [eic_dir, library_dir,mzml_dir,pl_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [286]:
matched_all = pd.DataFrame()
for mix in std_list_pos_adduct['mix'].unique():
    try:
        matched_temp = pd.read_csv(os.path.join(pl_dir, mix+'.csv'))
        matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    except:
        pass

In [120]:
eic_dir

'/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/EICs'

In [124]:
matched_all['rt_offset']=[abs(x-y) for x, y in zip(matched_all['reference_rt'], matched_all['rt_apex'])]
_tocheck = matched_all.query('rt_offset>0.1')
for index, row in tqdm(_tocheck.iterrows(), total = len(_tocheck)):
    if row['comment']=='Major':
        ms1, ms2 = read_mzml(row['reference_mix'], mzml_dir)
        rt_list, intensity_list = auto_EIC(row['reference_mz'],ms1)
        EIC(rt_list, intensity_list, vlines_location_1=[row['reference_rt']], vlines_location_2=[row['rt_apex']],
            savepath=os.path.join(eic_dir, row['reference_name']+'_'+row['reference_adduct']+'.png'), show=False)

100%|██████████| 48/48 [01:10<00:00,  1.47s/it]


In [127]:
msms_denoised = []
ei = []
for index, row in tqdm(matched_all.iterrows(), total = len(matched_all)):
    result_temp = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], reference_db_sorted, mass_error=0.02)
    msms_denoised.append(result_temp[0])
    ei.append(result_temp[1])
matched_all['ei']=ei
matched_all['msms_denoised']=msms_denoised

100%|██████████| 648/648 [02:10<00:00,  4.95it/s]


In [131]:
matched_good_quality

,precursor_mz,reference_mz,reference_adduct,reference_name,comment,reference_smiles,reference_mix,rt_apex,reference_rt,ms1_intensity,...,reci_snr,peak_purity,msms,ms1_scan_range,ms2_scan_idx_range_list,ms2_scan_idx,mix,reference_inchikey,ei,msms_denoised
0,717.143555,717.145011,[M+H]+,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-pos-mix1,3.017704,2.958,2.613462e+05,...,0.002060,1.000000,50.558895111083984\t2925.380859375\n53.9635696...,"[651, 658, 705]",[1979],1979,Polyphenols-pos-mix1,KMJPKUVSXFVQGZ-WQLSNUALSA-N,90.38914,53.96356964111328\t3832.4111328125\n139.038894...
1,611.310428,611.311563,[M+H]+,Daurisoline,Major,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,Polyphenols-pos-mix1,2.350485,2.311,3.160802e+07,...,0.001026,0.996047,54.24769592285156\t31611.298828125\n58.0650901...,"[505, 512, 556]","[1537, 1597, 1552, 1612, 1567, 1525, 1582]",1537,Polyphenols-pos-mix1,BURJAQFYNVMZDV-FIRIVFDPSA-N,95.39166,54.24769592285156\t31611.298828125\n58.0650901...
2,625.211778,625.212696,[M+H]+,Methyl-Hesperidin,Major,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,Polyphenols-pos-mix1,2.941834,2.882,9.698609e+06,...,0.000185,0.982593,54.1085205078125\t5332.501953125\n57.033763885...,"[635, 642, 679]","[1933, 1948, 1963, 1918]",1933,Polyphenols-pos-mix1,GUMSHIGGVOJLBP-SLRPQMTOSA-N,99.22022,54.1085205078125\t5332.501953125\n57.033763885...
3,642.238159,642.239244,[M+NH4]+,Methyl-Hesperidin,Major,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,Polyphenols-pos-mix1,2.941481,2.882,4.183011e+06,...,0.000017,0.997564,51.89076614379883\t3361.51171875\n57.033565521...,"[635, 642, 674]","[1925, 1940]",1925,Polyphenols-pos-mix1,GUMSHIGGVOJLBP-SLRPQMTOSA-N,99.27629,57.033565521240234\t5839.18896484375\n69.03336...
6,329.086006,329.086708,[M+H]+,Bergenin,Major,COc1c(O)cc2c(c1O)[C@@H]1O[C@H](CO)[C@@H](O)[C@...,Polyphenols-pos-mix1,1.656435,1.658,4.049463e+06,...,0.000031,0.987607,50.02085494995117\t2723.94189453125\n55.017810...,"[342, 362, 398]","[1060, 1117, 1075, 1090, 1045, 1102]",1090,Polyphenols-pos-mix1,YWJXCIXBAKGUKZ-HJJNZUOJSA-N,98.67315,50.02085494995117\t2723.94189453125\n55.017810...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,818.306771,818.307717,[M+NH4]+,Jionoside A1,Minor,COc1cc(/C=C/C(=O)O[C@@H]2[C@@H](CO[C@@H]3O[C@H...,Polyphenols-pos-mix65,2.353838,2.304,3.326095e+06,...,0.000198,0.992216,57.03371810913086\t4749.8623046875\n58.6339340...,"[511, 517, 548]","[1550, 1565]",1550,Polyphenols-pos-mix65,UAPZTGRENZINFN-WEDRDYHSSA-N,99.59151,57.03371810913086\t4749.8623046875\n69.0333328...
643,314.138050,314.138684,[M+H]+,Laetanine,Major,COc1cc2c(cc1O)-c1c(OC)c(O)cc3c1[C@H](C2)NCC3,Polyphenols-pos-mix66,1.991807,1.959,7.110285e+07,...,0.004438,0.961760,56.80146789550781\t50045.47265625\n61.74120330...,"[433, 442, 510]","[1441, 1396, 1354, 1309, 1501, 1456, 1411, 153...",1324,Polyphenols-pos-mix66,URQAEFLEDPPPFX-LBPRGKRZSA-N,97.66315,70.10330963134766\t50888.48828125\n74.12525177...
644,314.138149,314.138684,[M+H]+,Laetanine,isomer,COc1cc2c(cc1O)-c1c(OC)c(O)cc3c1[C@H](C2)NCC3,Polyphenols-pos-mix66,1.889438,1.959,9.765207e+05,...,0.141130,0.984798,56.8175163269043\t2944.499755859375\n62.653366...,"[412, 420, 429]","[1261, 1276, 1246]",1261,Polyphenols-pos-mix66,URQAEFLEDPPPFX-LBPRGKRZSA-N,97.14487,191.08609008789062\t4849.83642578125\n209.0958...
646,741.222722,741.223655,[M+H]+,Apigenin-7-O-(2G-rhamnosyl)gentiobioside,Major,CC1OC(OC2C(Oc3cc(O)c4c(=O)cc(-c5ccc(O)cc5)oc4c...,Polyphenols-pos-mix66,2.332191,2.285,6.016567e+06,...,0.000252,0.991558,57.03361129760742\t22052.11328125\n64.34982299...,"[509, 516, 552]","[1540, 1555, 1570]",1540,Polyphenols-pos-mix66,LXOPDILLGIDKLW-UHFFFAOYSA-N,98.17302,57.03361129760742\t22052.11328125\n69.03345489...


In [130]:
matched_good_quality = matched_all.query('ei>90 and rt_offset <=0.1')
matched_all.to_csv(os.path.join(library_dir, 'pos_matched_all.csv'), index = False)
matched_good_quality.to_csv(os.path.join(library_dir, 'pos_matched_good_quality.csv'), index = False)

In [139]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'pos_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)
export_library_msp(matched_all, os.path.join(library_dir, 'pos_matched_all.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 648/648 [00:00<00:00, 1637.45it/s]


In [288]:
matched_all=pd.read_csv(os.path.join(library_dir, 'pos_matched_all.csv'))
matched_good_quality = matched_all.query('ei>90')
rt_matched = []
for index, row in matched_good_quality.iterrows():
    if row['comment']=='Major' and row['rt_offset']>=0.1:
        rt_matched.append(False)
    else:
        rt_matched.append(True)
matched_good_quality['rt_matched']=rt_matched
matched_good_quality=matched_good_quality[matched_good_quality['rt_matched']==True]

In [290]:
matched_good_quality.to_csv(os.path.join(library_dir, 'pos_matched_good_quality.csv'), index = False)

In [291]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'pos_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 466/466 [00:00<00:00, 1622.38it/s]
